## Fine-tune Minerva3B-base 
To fine-tune Minerva3B-base we make use of the **unsloth** library. 
**NOTE**: The notebook was prepared to run on Google Colab. 

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import json
from huggingface_hub import HfApi, login
import torch
import os
from google.colab import drive
from datasets import load_from_disk#, load_dataset
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

To import `Minerva3B-base` you'll need to generate the token and log in to hugging-face.

In [ ]:
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)

#### Unsloth

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "sapienzanlp/Minerva-3B-base-v1.0",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)